## Contents 
     01. Importing libraries and data
     02. Data checks and preparation
     03. PII issues 
     04. Create flags for regions
     05. Spending and ordering habits across regions
     06. Create dataframe to show aggregate values for regions
     07. Excluding low frequency customers
     08. Export 

## Import libaries 

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## Import data

In [2]:
# setting path
path=r'C:\Users\alber\Instacart Basket Analysis'

In [3]:
# importing latest version of merged data
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_customer_merge.pkl'))

## Data checks

In [4]:
# checking dataframe 
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,product_id,add_to_cart_order,reordered,...,frequency_flag,first_name,last_name,gender,state,age,date_joined,number_of_dependents,marital_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [5]:
df.shape

(32404859, 34)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 34 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   order_id                          int64   
 1   user_id                           int64   
 2   order_number                      int64   
 3   orders_day_of_week                int64   
 4   order_hour_of_day                 int64   
 5   days_since_last_order             float64 
 6   new_customer                      bool    
 7   product_id                        int64   
 8   add_to_cart_order                 int64   
 9   reordered                         int64   
 10  product_name                      object  
 11  aisle_id                          int64   
 12  department_id                     int64   
 13  prices                            float64 
 14  _merge                            category
 15  price_range_loc                   object  
 16  busiest_day     

## Consider security implications

For security purposes, there's no need to have customer names. 'user_id' is the only thing necessary. 

In [7]:
# dropping first_name & last_name columns along with any columns I believe to be unnessary.
df2 = df.drop(columns=['first_name','last_name','_merge','aisle_id'])

In [8]:
# checking if columns were successfully dropped
df2.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,product_id,add_to_cart_order,reordered,...,spending_flag,median_days_since_previous_order,frequency_flag,gender,state,age,date_joined,number_of_dependents,marital_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0,...,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,False,196,1,1,...,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,False,196,1,1,...,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,False,196,1,1,...,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,False,196,1,1,...,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


## Creating regional flags

In [9]:
# creating regions with included states in regions
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 
             'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio',
          'North Dakota', 'South Dakota', 'Nebraska', 'Kansas',
          'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia',
         'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee',
         'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona',
        'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [10]:
# creating new region column
df2.loc[df2['state'].isin(Northeast),'Region'] = 'Northeast'

In [11]:
df2.loc[df2['state'].isin(Midwest),'Region'] = 'Midwest'

In [12]:
df2.loc[df2['state'].isin(South),'Region'] = 'South'

In [13]:
df2.loc[df2['state'].isin(West),'Region'] = 'West'

In [14]:
# checking all records were allocated to regions
df2['Region'].value_counts(dropna=False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: Region, dtype: int64

In [15]:
10791885+8292913+7597325+5722736

32404859

All records were allocated to a region successfully by cross-referencing with initial df.shape().

## Spending and ordering habits across regions

In [16]:
# grouping region and aggregating by customer_id 
df2[['Region', 'user_id']].groupby('Region').agg({'user_id': pd.Series.nunique,}).sort_values(by='user_id', ascending=False)

,user_id
Region,
South,68737
West,52565
Midwest,48519
Northeast,36388


According to the chart, the South has the most customers with the Northeast having the least amount of customers.

In [17]:
# checking which regions have the most sales
df2[['prices', 'Region']].groupby('Region').agg('sum').sort_values(by ='prices', ascending=False)

,prices
Region,
South,131613265.4
Midwest,96603658.7
West,93874398.7
Northeast,66126199.8


Similarly, the South has the most sales whereas the Northeast has the least amount of sales. However, despite the West having more customers, it is ranked below the Midwest in terms of sales.

In [18]:
# checking which regions have the most product sales
df2[['product_id', 'Region']].groupby('Region').agg('sum').sort_values(by ='product_id', ascending=False)

,product_id
Region,
South,276293783194
West,212188795772
Midwest,194569202091
Northeast,146469243221


Similar to the first chart, the South has the most product sales with the Northeast having the least amount of sales.

In [19]:
# use crosstab to see frequency of high and low spending customers in each region
cross = pd.crosstab(df2['Region'],df2['spending_flag'],dropna=False).sort_index()
cross

spending_flag,High spender,Low spender
Region,,
Midwest,7589534,7791
Northeast,5717129,5607
South,10781873,10012
West,8284433,8480


The highest frequency of high spending customers are in the South and the highest frequency of low spending customers in the Northeast.

In [20]:
# use crosstab to see the customer frequency in each region
cross2 = pd.crosstab(df2['Region'],df2['frequency_flag'],dropna=False).sort_index()
cross2

frequency_flag,Frequent customer,Non-frequent customer,Regular spender
Region,,,
Midwest,5017709,863419,1716197
Northeast,3804566,630182,1287988
South,7192372,1215088,2384420
West,5545206,927748,1819959


The South has the highest number of frequent customers, non-frequent customers, and regular spenders. The Northeast has the lowest number of frequent customers, non-frequent customers, and regular spenders.

In [21]:
# use cross tab to see customer loyalty in each region
cross3 = pd.crosstab(df2['Region'],df2['loyalty_flag'],dropna=False).sort_index()
cross3

loyalty_flag,Loyal customer,New customer,Regular customer
Region,,,
Midwest,2373774,1472573,3750978
Northeast,1841785,1100207,2780744
South,3405844,2074410,5311631
West,2662690,1596800,4033423


The South has the most loyal customers, new customers, and regular customers. The Northeast has the fewest loyal customers, new customers, and regular customers. 

## Creating dataframe to show aggregate values for regions

In [22]:
# creating new df to aggregate data by regions
# renaming columns to be more clarified
df_region = df2[['prices', 'Region', 'user_id', 'order_id']].groupby(['Region']).agg(
    {'prices':'sum', 'user_id': pd.Series.nunique, 'order_id': pd.Series.nunique})
df_region = df_region.rename(columns={'prices':'prices_sum', 'user_id': 'user_count', 'order_id': 'order_count'})
df_region

,prices_sum,user_count,order_count
Region,,,
Midwest,96603658.7,48519,754275
Northeast,66126199.8,36388,567162
South,131613265.4,68737,1071562
West,93874398.7,52565,821686


In [23]:
# creating calculated columns based on the aggregated data
df_region['price_per_order'] = df_region['prices_sum']/df_region['order_count']
df_region['price_per_user'] = df_region['prices_sum']/df_region['user_count']
df_region['order_per_user'] = df_region['order_count']/df_region['user_count']

In [24]:
df_region.sort_values(by='price_per_user',ascending=False)

,prices_sum,user_count,order_count,price_per_order,price_per_user,order_per_user
Region,,,,,,
Midwest,96603658.7,48519,754275,128.074852,1991.048016,15.545972
South,131613265.4,68737,1071562,122.823752,1914.736829,15.589304
Northeast,66126199.8,36388,567162,116.591379,1817.252935,15.586512
West,93874398.7,52565,821686,114.246073,1785.872704,15.631808


## Excluding low frequency customers 

In [25]:
# creating variable for orders made by each user. The count of user_id would be the number of orders user has made.
df2['total_orders']=df2.groupby(['user_id']).user_id.transform('count')

In [26]:
# creating exclusion flag using total_orders. If total_orders > 5, then would be considered low activity.
results_2=[]
for total in df2['total_orders']:
    if total < 5:
        results_2.append('Low Activity')
    else:
        results_2.append('Regular Activity')

In [27]:
# including column in dataframe
df2['activity_flag']=results_2

In [28]:
# checking dataframe for low activity customers
df2['activity_flag'].value_counts(dropna=False)

Regular Activity    32401975
Low Activity            2884
Name: activity_flag, dtype: int64

In [29]:
# creating subset of low activity customers
df3=df2[df2['activity_flag'] == 'Low Activity']

In [30]:
# checking if creating subset was successful
df3

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,product_id,add_to_cart_order,reordered,...,gender,state,age,date_joined,number_of_dependents,marital_status,income,Region,total_orders,activity_flag
37671,1384521,6206,3,3,10,29.0,False,196,1,0,...,Female,Michigan,40,10/28/2018,1,married,30652,Midwest,4,Low Activity
37672,3381981,6206,1,1,12,NaN,True,21573,1,0,...,Female,Michigan,40,10/28/2018,1,married,30652,Midwest,4,Low Activity
37673,2058522,6206,2,2,11,15.0,False,39108,2,0,...,Female,Michigan,40,10/28/2018,1,married,30652,Midwest,4,Low Activity
37674,2058522,6206,2,2,11,15.0,False,40688,1,0,...,Female,Michigan,40,10/28/2018,1,married,30652,Midwest,4,Low Activity
57726,2534676,9346,1,1,12,NaN,True,196,1,0,...,Female,Idaho,42,1/9/2018,2,married,149794,West,4,Low Activity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404828,1947537,166751,3,4,7,0.0,False,49001,1,1,...,Female,Oregon,38,8/17/2018,1,married,41766,West,3,Low Activity
32404855,484769,66343,1,6,11,NaN,True,47210,1,0,...,Female,Tennessee,22,9/12/2017,3,married,46151,South,4,Low Activity
32404856,1561557,66343,2,1,11,30.0,False,47210,1,1,...,Female,Tennessee,22,9/12/2017,3,married,46151,South,4,Low Activity
32404857,276317,66343,3,6,15,19.0,False,47210,1,1,...,Female,Tennessee,22,9/12/2017,3,married,46151,South,4,Low Activity


We see that df3 contains 2884 rows containing 'Low Activity'.

In [34]:
# export low activity subset
df3.to_csv(os.path.join(path, '02 Data','Prepared Data', 'low_activity_customers.csv'))

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\alber\\Instacart Basket Analysis\\02 Data\\Prepared Data\\low_activity_customers.csv'

In [35]:
# excluding sample from dataset
df2.drop(df2[df2['activity_flag']=='Low Activity'].index, inplace=True)

In [36]:
# checking to see if sample was dropped
df2['activity_flag'].value_counts()

Regular Activity    32401975
Name: activity_flag, dtype: int64

In [37]:
# viewing dataset 
pd.set_option('display.max_columns', None)
df2.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,product_id,add_to_cart_order,reordered,product_name,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_prices,spending_flag,median_days_since_previous_order,frequency_flag,gender,state,age,date_joined,number_of_dependents,marital_status,income,Region,total_orders,activity_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,7,9.0,Mid range product,Regularly busy,Regularly busy,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,7,9.0,Mid range product,Regularly busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,7,9.0,Mid range product,Regularly busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,7,9.0,Mid range product,Least busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,7,9.0,Mid range product,Least busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity


In [39]:
# creating subset of regular activity customers
df4=df2[df2['activity_flag'] == 'Regular Activity']
df4()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,product_id,add_to_cart_order,reordered,product_name,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_prices,spending_flag,median_days_since_previous_order,frequency_flag,gender,state,age,date_joined,number_of_dependents,marital_status,income,Region,total_orders,activity_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,7,9.0,Mid range product,Regularly busy,Regularly busy,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,7,9.0,Mid range product,Regularly busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,7,9.0,Mid range product,Regularly busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,7,9.0,Mid range product,Least busy,Slowest days,Average orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,7,9.0,Mid range product,Least busy,Slowest days,Most orders,10,New customer,14.0,High spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity


In [40]:
df4.shape

(32401975, 33)

In [41]:
df4.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'reg_activity_customers.pkl'))